In [ ]:
import sys
sys.path.append('../')
sys.path.append('C:\\Users\\olegk\\Documents\\unifloc\\')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import datetime
from datetime import date
from os import walk
import os

import uniflocpy.uTools.plotly_workflow as pw
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, plot

In [ ]:
# TODO проверить первичную обработку данных путем построения графиков
# TODO разобраться с ошибками при первичной обработке данных
# TODO где среднее значение, а где протяжка
# TODO генерация входных данных для адаптации и восстановление отдельно (drop и resample, resample и interpolate)
# TODO резать данные кусками в связи с пропусками
# TODO сделать .py и работать с дебагом
# TODO обрезать данные, чтобы отрефакторить обработку (первичную)
# TODO протягивать замеры (в шахматке)
# TODO борьба с выбросами

In [ ]:
def plot_corr_heatmap(df, filename_str):
    corrs = df.corr()
    figure = ff.create_annotated_heatmap(
       z=corrs.values,
       x=list(corrs.columns),
       y=list(corrs.index),
       annotation_text=corrs.round(2).values,
       showscale=True)
    figure.layout.update(
        margin=go.layout.Margin(
            l=400,
            r=50,
            b=100,
            t=250
        ))
    filename_str +='.html'
    plot(figure,filename=filename_str)
    
def initial_editing(data, wellname):
    if len(data.columns)==4:  
        del data[0]
    data.columns = [0, 1, 2]
    test_str = data[0][0]
    index = test_str.find(wellname)
    str_to_delete = test_str[:index] + wellname + '. '
    data[0] = data[0].str.replace(str_to_delete,"")
    data.index = data[1]
    del data[1]
    data.index = pd.to_datetime(data.index)
    return data

def extract_df_one_parametr_and_edit(data, list_of_params, number_of_param_in_list):
    extracted_df_one_param = data[data[0] == list_of_params[number_of_param_in_list]].copy()
    edited_df_one_param = extracted_df_one_param.rename(index=str, columns = {2: extracted_df_one_param[0][0]})
    del edited_df_one_param[0]
    return edited_df_one_param

def create_edited_df(data, parametrs_list):  # ускорить процесс
    init_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, 0)
    result = init_one_parametr_df
    for i in range(1, len(parametrs_list)):
        new_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, i)
        result = result.join([new_one_parametr_df], how = "outer", sort=True)
    return result


name_columns = {"Qж ТМ": "Qж ТМ (Ш)",
               "Qн ТМ": "Qн ТМ (Ш)",
               "Обв ТМ": "Обв ТМ (Ш)",
               "Рбуф ТМ": "Рбуф ТМ (Ш)",
                "Рбуф": "Рбуф (Ш)",
                "Рэцн  ТМ": "Рэцн  ТМ (Ш)",
                "Qгаз ТМ": "Qгаз ТМ (Ш)",
               "Рлин ТМ": "Рлин ТМ (Ш)",
               "Рзатр": "Рзатр (Ш)",
               "Рприем": "Рприем (Ш)",
               "Загр. ПЭД (ТМ)": "Загр. ПЭД (ТМ) (Ш)",
               "Нд": "Нд (Ш)",
               "Темп. ПЭД": "Темп. ПЭД (Ш)",
               "Тдвиг ТМ": "Тдвиг ТМ (Ш)",
               "F вращ ТМ": "F вращ ТМ (Ш)",
               "Dшт": "Dшт (Ш)",
               "F ТМ": "F ТМ (Ш)",
               "U AB (ТМ)": "U AB (ТМ) (Ш)",
               "I": "I (Ш)",
               "Tуст. (ТМ)": "Tуст. (ТМ) (Ш)",
               "T на приёме нас.": "T на приёме нас. (Ш)",
               "Темп. ж. ТМ": "Темп. ж. ТМ (Ш)",
               "Тнасос ТМ": "Тнасос ТМ (Ш)",
               "I A ТМ": "I A ТМ (Ш)",
               "I C ТМ": "I C ТМ (Ш)",
               "I В ТМ": "I В ТМ (Ш)",
               "R изол ТМ": "R изол ТМ (Ш)",
               "R изоляц.": "R изоляц. (Ш)",
               "U раб.": "U раб. (Ш)",
                
               "Активная мощность": "Активная мощность (СУ)",
               "Выходная частота ПЧ": "Выходная частота ПЧ (СУ)",
               "Давление на приеме насоса (пласт. жидкость)": "Давление на приеме насоса (пласт. жидкость) (СУ)",
               "Загрузка двигателя": "Загрузка двигателя (СУ)",
               "Коэффициент мощности": "Коэффициент мощности (СУ)",
               "Массовый дебит жидкости": "Массовый дебит жидкости (СУ)",
               "Массовый дебит нефти": "Массовый дебит нефти (СУ)",
               "Напряжение на выходе ТМПН": "Напряжение на выходе ТМПН (СУ)",
               "Объемный дебит воды": "Объемный дебит воды (СУ)",  
               "Объемный дебит газа": "Объемный дебит газа (СУ)",
               "Объемный дебит жидкости": "Объемный дебит жидкости (СУ)",
               "Объемный дебит нефти": "Объемный дебит нефти (СУ)",
               "Полная мощность": "Полная мощность (СУ)",
               "Процент обводненности": "Процент обводненности (СУ)",
               "Температура на приеме насоса (пласт. жидкость)": "Температура на приеме насоса (пласт. жидкость) (СУ)",
               "ГФ": "ГФ (СУ)",
        
               "Pline_atma": "P лин., атм (Модель, вход)",
               "pbuf_atma": "P буф., атм (Модель, выход, адапт. Ш)",
               "Pdis_atma": "P выкид ЭЦН, атм (Модель, выход)",
               "Pint_atma": "P прием ЭЦН, атм (Модель, вход)",
               "pwf_atma": "P прием ЭЦН (P заб. модели) атм (Модель, вход)",
               "qliq_sm3day": "Q ж, м3/сут (Модель, вход)",
               "fw_perc": "Обв, % (Модель, вход)",
               "tbh_C": "T прием, С (Модель, вход)",
               "twh_C": "T устья, С (Модель, выход)",
               "Tsurf_C": "T поверхности, С (Модель, вход)",
               "t_dis_C": "T выкид ЭЦН, С (Модель, выход)",
               "Tintake_C": "T прием ЭЦН, C (Модель, вход)",
               "choke.c_degrad_fr": "Коэффициент деградации для штуцера, ед (Модель, вход)",
               "Pcas_atma": "P затрубное, атм (Модель, вход)",
               "Hdyn_m": "H дин.ур., м (Модель, выход)",
               "ESP.power_CS_calc_W": "Акт. мощность на СУ, (Модель, выход, адапт. СУ)",
               "ESP.freq_Hz": "F тока, ГЦ (Модель, вход)",
               "ESP.I_A": "I, А (Модель, выход)",
               "ESP.U_V": "U, В (Модель, вход, СУ)",
               "ESP.load_fr": "Загрузка двигателя, (Модель, выход)",
               "ESP.ESPpump.EffiencyESP_d": "КПД ЭЦН, д.ед. (Модель, выход)",
               "ESP.KsepTotal_fr": "К сеп. общий, д.ед. (Модель, выход)",
               "ESP.KsepNat_fr": "К сеп. естесственной, д.ед. (Модель, выход)",
               "ESP.KSepGasSep_fr": "К сеп. газосепаратора (Модель, вход)",
               "ESP.c_calibr_head": "Коэффициент калибровки по напору - множитель (Модель, вход)",
                "ESP.c_calibr_power": "Коэффициент калибровки по мощности - множитель (Модель, вход)",
                "ESP.c_calibr_rate": "Коэффициент калибровки по дебиту - множитель (Модель, вход)",
                "ESP.power_motor_nom_W": "Номинальная мощность ПЭД, Вт (Модель, вход)",
                "ESP.cable_dU_V": "dU в кабеле, В (Модель, выход)",
                "ESP.dPower_GasSep_W": "Мощность, потребляемая газосепаратором (Модель, выход)",
                "ESP.dPower_protector_W": "Мощность, потребляемая протектором (Модель, выход)",
                "ESP.cable_dPower_W": "Мощность, потребляемая (рассеиваемая) кабелем (Модель, выход)",
                "ESP.dPower_transform_W": "Мощность, потребляемая ТМПН (Модель, выход)",
                "ESP.dPower_CS_W": "Мощность, потребляемая СУ (Модель, выход)",
                "ESP.ESPpump.Powerfluid_Wt": "Мощность, передаваемая жидкости (Модель, выход)",
                "ESP.ESPpump.PowerESP_Wt": "Мощность, передаваемая ЭЦН (Модель, выход)",
                "ESP.power_shaft_W": "Мощность, передаваемая валу от ПЭД (Модель, выход)",
                "ESP.power_motor_W": "Мощность, передаваемая ПЭД (Модель, выход)",
                "ESP.cable_power_W": "Мощность, передаваемая кабелю (Модель, выход)",
                "ESP.power_CS_teor_calc_W": "Мощность, передаваемая СУ (Модель, выход,  адапт. СУ)",
                "calibr_multiply": "Произведение калибровок по мощности и напору (Постобработка)"
               }
columns_to_delete = ["T поверхности, С (Модель, вход)",
                    "Мощность, потребляемая газосепаратором (Модель, выход)",
                    "Мощность, потребляемая протектором (Модель, выход)",
                    "Номинальная мощность ПЭД, Вт (Модель, вход)",
                    "P затрубное, атм (Модель, вход)",
                     "H дин.ур., м (Модель, выход)",
                     "К сеп. газосепаратора (Модель, вход)",
                     "Коэффициент калибровки по дебиту - множитель (Модель, вход)",
                     "Коэффициент деградации для штуцера, ед (Модель, вход)",
                     "F тока, ГЦ (Модель, вход)"
                     
                    ]

columns_to_choice = ["Коэффициент калибровки по напору - множитель (Модель, вход)",
                     "Коэффициент калибровки по мощности - множитель (Модель, вход)",
                     "Произведение калибровок по мощности и напору (Постобработка)",
                     "Мощность, передаваемая СУ (Модель, выход,  адапт. СУ)",
                     
                     "К сеп. общий, д.ед. (Модель, выход)",
                    "КПД ЭЦН, д.ед. (Модель, выход)",
                     "Q ж, м3/сут (Модель, вход)",
                     "ГФ (СУ)",
                     "Обв, % (Модель, вход)",
                     "P буф., атм (Модель, выход, адапт. Ш)",
                     "P прием ЭЦН, атм (Модель, вход)",
                    ]

columns_to_careful_analisys = ["Q ж, м3/сут (Модель, вход)",
                               "Коэффициент калибровки по напору - множитель (Модель, вход)",
                             "Коэффициент калибровки по мощности - множитель (Модель, вход)",
                               "Мощность, передаваемая СУ (Модель, выход,  адапт. СУ)"
                               
                              ]

def prepare_to_corr_analys(df, columns_to_delete = columns_to_delete):
    new_df = df.copy()
    for i in columns_to_delete:
        del new_df[i]
    return new_df
def rename_columns(df, columns_name_dict = name_columns):
    for i in df.columns:
        if i in columns_name_dict.keys():
            df = df.rename(columns={i: name_columns[i] })
    return df


def prepare_df(df):
    columns_to_delete = ['d', 'Unnamed: 0', 'Unnamed: 42', 'Unnamed: 39']
    for i in  columns_to_delete:
        if i in df.columns:
            del df[i]
    #df = df.iloc[1:]
    return df

Получение списка скважин по названию соответствующих директорий

In [ ]:
current_path = os.getcwd()
path_to_data = current_path + "\\data\\"
dirnames_list = []
for (dirpath, dirnames, filenames) in walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)

In [ ]:
#files = dirnames_list
files = ['353']
test_file = None

Первичная обработка данных, разнесение параметров по столбцам

In [ ]:
first_edit = False
if first_edit == True:
    for this_file_name in files:
        print("Первичная обработка данных со СУ для скважины " + this_file_name)

        this_file_path = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + ".csv"
        test_file = pd.read_csv(this_file_path,sep=';', header=None)
        if type(test_file.iloc[0][2]) == str:
            test_file = pd.read_csv(this_file_path,sep=';', header=None, skiprows = 1, low_memory=False)
        edited_file = initial_editing(test_file, this_file_name)
        big_list_parametrs = np.unique(edited_file[0])
        edited_file = create_edited_df(edited_file, big_list_parametrs)
        this_file_path_out = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + "_first_edit" +  ".csv"
        print("Сохранение обработанных данных в:")
        print(this_file_path_out)
        edited_file.to_csv(this_file_path_out)

Скважины с ошибками

In [ ]:
error_wells = ['689','627', '202', '353']

Объденение данных с шахматки и со СУ

In [ ]:
well = '601'
this_file_name = 'Скв. 601 (01.07.2018-31.03.2019)'
make_restore_input = True
start = [datetime.datetime(2018,8,3), datetime.datetime(2018,11,29)]
end = [datetime.datetime(2018,11,6), datetime.datetime(2019,2,28)]
calirations_file_name = 'Коэффициенты калибровки 601 время 2019_10_28_14_44_05'

In [ ]:
if make_restore_input:
    restore_mark = '_restore_'
else:
    restore_mark = ''
this_file_path = current_path + "\\data\\" + well +  "\\" + this_file_name + ".xls"
print(this_file_path)
test_file = pd.read_excel(this_file_path)
test_file.index = pd.to_datetime(test_file['Дата'], dayfirst = True, format = "%d.%m.%Y", infer_datetime_format=True)
del test_file['Дата']
test_file = test_file[test_file.columns[5:]]
test_file = test_file.resample('3h').last()
test_file = test_file.fillna(method='ffill')
path_to_chess_out = current_path + "\\data\\" + well +  "\\"  + well + 'шахматка обработанная' + ".csv"
print(path_to_chess_out)

test_file.to_csv(path_to_chess_out)
all_traces = pw.create_traces_list_for_all_columms(test_file, chosen_mode = 'lines+markers')

data = all_traces
path_to_plot = current_path + "\\data\\" + well +  "\\" + \
            'шахматка обработанная ' + well +  '.html'
print("Сохранение графика в:")
print(path_to_plot)
pw.plot_subplots(data, path_to_plot, True)

this_file_path = current_path + "\\data\\" + well +  "\\" + well + "_first_edit" +  ".csv"
try:
    test = pd.read_csv(this_file_path, index_col = '1')
except:
    test = pd.read_csv(this_file_path)
    test = test.rename(columns={"Unnamed: 0": "1"})
    test.index = test["1"]
test.index = pd.to_datetime(test.index)
test_resampled = test.resample('3h').mean()
test_resampled['Температура на приеме насоса (пласт. жидкость)'] = test_resampled['Температура на приеме насоса (пласт. жидкость)'].fillna(method='ffill')
all_data = test_file.join(test_resampled, how='outer')
if make_restore_input:
    all_data = all_data.fillna(method='ffill')
    calirations_file_path = current_path + "\\data\\" + well +  "\\" +calirations_file_name + '.csv'
    calibrations = pd.read_csv(calirations_file_path)
    calibrations.index = pd.to_datetime(calibrations['Unnamed: 0'])
    del calibrations['Unnamed: 0']
    calibrations = calibrations[calibrations['Коэффициент калибровки по напору - множитель (Модель, вход)'] > 0.505]
    calibrations = calibrations.resample('3h').last()
    calibrations = calibrations.interpolate() #
    all_data = all_data.join(calibrations, how = "inner", sort=True)

    restore_df = []
    for i in range(len(start)):
        new_df = all_data[(all_data.index >= start[i]) & (all_data.index <= end[i])]
        restore_df.append(new_df)
    result = restore_df[0]
    for j in range(1, len(restore_df)):     
        result = result.append(restore_df[j])
    result = result.fillna(method='ffill')
    all_data = result
else:
    all_data = all_data.dropna(subset = ['Объемный дебит жидкости'])
all_data['ГФ'] = all_data['Объемный дебит газа'] / all_data['Объемный дебит нефти']
all_data = all_data.fillna(method='ffill')
all_data = rename_columns(all_data)
all_data = all_data[all_data['Выходная частота ПЧ (СУ)'] > 0]
all_traces = pw.create_traces_list_for_all_columms(all_data, chosen_mode = 'lines+markers')
data = all_traces
path_to_plot = current_path + "\\data\\" + well +  "\\" + \
            'все данные сведенные Ш и СУ 3 часа только Q ж ' + well + restore_mark +    '.html'
print("Сохранение графика в:")
print(path_to_plot)
pw.plot_subplots(data, path_to_plot, True)

path_to_input_data_out = current_path + "\\data\\" + well +  "\\"  + well + restore_mark + '_input_data' + ".csv"
all_data.to_csv(path_to_input_data_out)

Построение графиков для всех скважин

In [ ]:
#for i in files:
plot_plots = False
if plot_plots == True:
    for i in ['1509']:
        if i not in error_wells:
            print("Построение графиков для скважины " + i)
            well = i
            this_file_path = current_path + "\\data\\" + well +  "\\" + well + "_first_edit" +  ".csv"
            try:
                test = pd.read_csv(this_file_path, index_col = '1')
            except:
                test = pd.read_csv(this_file_path)
                test = test.rename(columns={"Unnamed: 0": "1"})
                test.index = test["1"]
            test.index = pd.to_datetime(test.index)
            test_resampled = test.resample('3h').mean()

            test_resampled = test_resampled.dropna(subset = ['Объемный дебит жидкости'])

            all_traces = pw.create_traces_list_for_all_columms(test_resampled, chosen_mode = 'lines+markers')

            data = all_traces
            path_to_plot = current_path + "\\data\\" + well +  "\\" + \
                        'Исходные данные (first_edit) по 3 часа без интерполяции для ' + well +  '.html'
            print("Сохранение графика в:")
            print(path_to_plot)
            pw.plot_subplots(data, path_to_plot, True)
            #plot_corr_heatmap(test_resampled, "Корреляционная карта для данных по 3 часа без интерполяции")